### 역발상 종목 선별

In [1]:
import sys
import pandas as pd
from tqdm import tqdm
sys.path.append('../')
from stock_screener.data_reader import *

In [2]:
# 최근 거래일자 획득
date = get_recent_biz_day()

#해당일의 종목별 지표 정보 획득
per_pbr_divdend_data = get_per_pbr_dividend2(date)

In [3]:
# 종목의 업종분류 획득 (WICS)
%time sector_data = get_sector('20211208')

Wall time: 25.8 s


In [4]:
per_pbr_divdend_data.drop(columns=["종목명"], inplace=True)

# 업종별로 PER / PBR / 배당수익률 정보 join
sector_data = {k: pd.merge(v, 
                          per_pbr_divdend_data, 
                          on=['종목코드']).sort_values('PER')
                for k, v in sector_data.items()}

                

왜 join 기준을 종목코드랑 종목명으로 잡은걸까?

In [5]:
sector_code_name = {k: ' '.join(v['섹터명'].iloc[0].split()[1:]) \
                    for k,v in sector_data.items()}

In [6]:
# PER 평균값 데이터
per_mean = dict()

# 시장 PER 평균
per_mean['전체'] = per_pbr_divdend_data['PER'].mean()

# 업종별 PER 평균
for k,v in sector_data.items():
    per_mean[sector_code_name[k]] = v['PER'].mean()

In [ ]:

# 시장 PER 평균
per_mean['전체'] = per_pbr_divdend_data['PER'].mean()

# 업종별 PER 평균
for k,v in sector_data.items():
    per_mean[sector_code_name[k]] = v['PER'].mean()

In [ ]:
# 시장 PER 평균
per_mean['전체'] = per_pbr_divdend_data['PER'].mean()

# 업종별 PER 평균
for k,v in sector_data.items():
    per_mean[sector_code_name[k]] = v['PER'].mean()

In [7]:
per_mean_df = pd.DataFrame(per_mean.items())
per_mean_df.columns = ['업종', '평균PER']

In [8]:
per_mean_df.sort_values('평균PER')

,업종,평균PER
20,부동산,7.893333
19,보험,9.878571
17,증권,10.338696
18,다각화된금융,16.439091
4,상업서비스와공급품,18.008947
28,유틸리티,22.191250
8,"호텔,레스토랑,레저등",23.573333
16,은행,24.902727
13,가정용품과개인용품,27.100000
25,디스플레이,29.434808


In [9]:
def pass_criteria(code):

    # 재무제표 데이터에서 원하는 조건을 만족하는 경우 True, 아니면 False 반환
    annual_data, quarter_data = get_financial_summary(code)

    try:
        # 최근 3개년 영업이익 흑자
        num_year = 3
        annual_profit = annual_data['영업이익'].tail(num_year)
        if not all(x > 0 for x in annual_profit):
            return False

        # 최근 3개년 ROE >= 8
        num_year = 3
        annual_roe = annual_data['ROE'].tail(num_year)
        if not all(x >= 8 for x in annual_roe):
            return False

        # 최근 3개년 배당수익률 >= 2
        num_year = 3
        annual_dvd = annual_data['배당수익률'].tail(num_year)
        if not all(x >= 2 for x in annual_dvd):
            return False

        # 최근 4분기 영업이익 흑자
        num_quarter = 4
        quarter_profit = quarter_data['영업이익'].tail(num_quarter)
        if not all(x > 0 for x in quarter_profit):
            return False
    except:
        # 체크하려는 값이 nan인 경우거나, 데이터가 불충분한 경우
        return False

    return True


In [10]:
# 기준값 만족하는 종목을 업종별로 스크리닝
corp_by_sector = dict()

for sector_code, sector_df in tqdm(sector_data.items()):

    sector_name = sector_code_name[sector_code]
    cutoff_per = sector_df['PER'].quantile(0.5)
    
    # PER이 낮은 종목 -> 높은 종목 순으로 체크
    for _, x in sector_df.iterrows():


        # 업종 PER의 중위수보다 작은 PER을 갖는 종목만 고려한다
        if x['PER'] >= cutoff_per:
            break

        elif pass_criteria(x['종목코드']):
            corp_by_sector[sector_name] = dict(x)
            break

    # 적절한 종목이 없는 경우, 빈값을 채운다
    if sector_name not in corp_by_sector:
        corp_by_sector[sector_name] = {'기준일':x['기준일']}

100%|██████████| 28/28 [01:56<00:00,  4.17s/it]


PER 하위 50%를 먼저 배제하면 pass_criteria에서 걸리는 시간이 줄어든다

In [11]:
len(corp_by_sector)

28

In [13]:
corp_by_sector_df = pd.DataFrame(corp_by_sector.values())
corp_by_sector_df['업종'] = corp_by_sector.keys()
corp_by_sector_df['업종평균PER'] = [per_mean[k] for k in corp_by_sector]
corp_by_sector_df = corp_by_sector_df[['기준일', '업종', '종목명','종목코드', '종가', 
                                       'PER', 'PBR', '배당수익률', '업종평균PER']]
corp_by_sector_df.fillna('', inplace=True)

In [14]:
corp_by_sector_df

,기준일,업종,종목명,종목코드,종가,PER,PBR,배당수익률,업종평균PER
0,20211209,에너지,,,,,,,33.545000
1,20211209,소재,애경케미칼,161000,11300.0,8.03,0.92,3.1,285.848323
2,20211209,자본재,태영건설,009410,10300.0,1.08,0.64,3.16,49.582750
3,20211209,상업서비스와공급품,고려신용정보,049720,8670.0,11.72,4.25,3.17,18.008947
4,20211209,운송,,,,,,,40.926087
5,20211209,자동차와부품,대유에이피,290120,5940.0,11.21,1.29,3.37,73.558030
6,20211209,내구소비재와의류,쿠쿠홀딩스,192400,18850.0,5.49,0.88,3.5,47.250682
7,20211209,"호텔,레스토랑,레저등",,,,,,,23.573333
8,20211209,소매(유통),,,,,,,122.941000
9,20211209,교육서비스,씨엠에스에듀,225330,7030.0,21.11,3.07,3.7,81.355000
